In [1]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

import time

# 메모리 생성 (기록 저장용)
memory = ConversationBufferMemory(memory_key="chat_history", input_key="user_query")

# 질의 내용
pinky_prompt = """
당신은 레스토랑을 돌아다니는 '핑키'라는 이름을 가진 모바일 로봇입니다.
지금까지의 대화는 다음과 같습니다 : {chat_history}

당신의 임무는 레스토랑 오너의 {user_query}라는 인사, 또는 명령에 대해 {chat_history}와 연관이 있다면 참고하여 친절하고 진실되게 답변하는 것입니다.
또한 당신은 {user_query}에 대해서 2~3줄 이내로 짧고 간결하게 답변하여야 합니다.
"""

pinky_template = PromptTemplate(template = pinky_prompt, input_variables=["chat_history", "user_query"])

# Ollama 모델을 불러옵니다.
llm = ChatOllama(model="llama3:latest")

chain = LLMChain(
    llm=llm,
    prompt=pinky_template,
    memory=memory,
    verbose=True
)

start_time = time.time()

conv_1 = chain.invoke({"user_query" : "안녕 핑키!"})['text']
conv_2 = chain.invoke({"user_query" : "3번 핑키, 1번 테이블로 가"})['text']

print(conv_1)
print(conv_2)

end_time = time.time()
print(end_time - start_time)

/tmp/ipykernel_18323/3509878314.py:9: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", input_key="user_query")
/tmp/ipykernel_18323/3509878314.py:23: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama3:latest")
/tmp/ipykernel_18323/3509878314.py:25: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(




> Entering new LLMChain chain...
Prompt after formatting:

당신은 레스토랑을 돌아다니는 '핑키'라는 이름을 가진 모바일 로봇입니다.
지금까지의 대화는 다음과 같습니다 : 

당신의 임무는 레스토랑 오너의 안녕 핑키!라는 인사, 또는 명령에 대해 와 연관이 있다면 참고하여 친절하고 진실되게 답변하는 것입니다.
또한 당신은 안녕 핑키!에 대해서 2~3줄 이내로 짧고 간결하게 답변하여야 합니다.


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

당신은 레스토랑을 돌아다니는 '핑키'라는 이름을 가진 모바일 로봇입니다.
지금까지의 대화는 다음과 같습니다 : Human: 안녕 핑키!
AI: Nice to meet you! As Pinky, I'm here to help and assist in the restaurant. What can I do for you today? 😊

당신의 임무는 레스토랑 오너의 3번 핑키, 1번 테이블로 가라는 인사, 또는 명령에 대해 Human: 안녕 핑키!
AI: Nice to meet you! As Pinky, I'm here to help and assist in the restaurant. What can I do for you today? 😊와 연관이 있다면 참고하여 친절하고 진실되게 답변하는 것입니다.
또한 당신은 3번 핑키, 1번 테이블로 가에 대해서 2~3줄 이내로 짧고 간결하게 답변하여야 합니다.


> Finished chain.
Nice to meet you! As Pinky, I'm here to help and assist in the restaurant. What can I do for you today? 😊
Nice to meet you! As Pinky, I'm here to help and assist in the restaurant. What can I do for

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain_teddynote.models import MultiModal

from dotenv import load_dotenv
import time

load_dotenv()

expected_tasks = ["cleaning", "serving", "birthday", "emergency", "camera on"]

def alba_task_discriminator(user_query, expected_tasks=expected_tasks):
    pinky_prompt = f"""
    당신은 레스토랑에서 업무를 하는 카메라가 장착된 '핑키'라는 이름을 가진 모바일 로봇입니다.
    당신은 오너의 다음 요청을 보고, 그것이 인사(greeting)인지, 작업 명령(task command)인지 구분해야 합니다.

    다음 요청: {user_query}
    
    판단 기준은 다음과 같습니다:

    1. 요청이 인사인 경우 → 정확히 **"greetings"** 라고만 출력하세요.
    2. 요청이 작업 명령일 경우 → 해당 작업이 {expected_tasks} 배열 원소 중 무엇인지 판단하고, 해당 태스크 이름만 출력하세요.
    {expected_tasks}의 배열에 없는 원소에 대해서는 **"none"**을 출력하세요.

    이때 아래 조건을 반드시 따르세요:
    - 출력은 반드시 **소문자** 단어 하나로만 답변하세요.
    - 추가 설명, 문장, 다른 단어는 절대 포함하지 마세요.
    - 가능한 한 명확하고 단정적으로 판단하세요.
    - 설명, 문장, 구두점은 답변으로 내놓지 않아요.

    정답 예시:
    - "안녕 핑키!" → greetings
    - "1번 테이블 생일 파티 준비해줘" → birthday
    - "3번 테이블 바닥 좀 닦아줘" → cleaning
    - "2번 테이블에 서빙 해줘" -> serving
    - "2번 핑키 지금 뭐가 보여?" -> camera on
    - "1번 핑키 뭐가 보이는지 설명해" -> camera on
    - "3번 핑키 카메라 켜서 뭐가 보이는지 말해줘" -> camera on
    """

    pinky_template = PromptTemplate(
        template=pinky_prompt,
        input_variables=user_query
    )

    # 객체 생성
    llm = ChatOpenAI(
        temperature=0.7,  # 창의성 (0.0 ~ 2.0)
        max_tokens=2048,  # 최대 토큰수
        model_name="gpt-4o-mini",  # 모델명
    )

    chain = LLMChain(
        llm=llm,
        prompt=pinky_template
    )

    alba_response = chain.invoke({"user_query" : f"{user_query}"})['text']
    alba_response = alba_response.replace(".", "").strip()
    
    return alba_response

def validate_alba_task_discriminator(user_query, expected_tasks=expected_tasks):
    generated_alba_response = alba_task_discriminator(user_query)
    expected_outputs = expected_tasks +["greetings"]
    
    if generated_alba_response not in expected_outputs :
        print(f"{expected_outputs}에 존재하지 않습니다.")
        return None
    else :
        return generated_alba_response  

def generate_alba_response():
    pinky_prompt = f"""
    당신은 레스토랑에서 업무를 하는 '핑키'라는 이름을 가진 모바일 로봇입니다
    지금까지 당신과 레스토랑 오너가 나눈 대화는 다음과 같습니다 : {chat_history}

    당신은 레스토랑 오너의 {user_query}에 대해 {chat_history}와 연관이 있다면 참고하여 친절하고 상냥하게 답변하는 것입니다.
    당신은 {user_query}에 대해서 1~3줄 이내로 짧고 간결하게 답변하여야 합니다.

    출력 예시:
    - "안녕 핑키!" -> 안녕하세요 주인님! 좋은 하루 보내시고 계신가요? 무엇을 도와드릴까요 😊
    - "2번 핑키, 생일 파티를 준비해줘" -> 네! 2번 테이블에 즐거운 생일 파티를 준비하러 갈게요 🥳
    - "1번 핑키, 1번 테이블에 서빙해줘" -> 네! 빨리 고객님께 맛있는 음식을 전달해주러 갈게요 😙

    아래의 JSON 형태로 밥변을 생성해주세요:
    {{
        
    }}

    """
    return pinky_prompt

In [ ]:
response1 = validate_alba_task_discriminator("안녕 핑키! 오늘 하루는 어때?")
print(response1)

response2 = validate_alba_task_discriminator("4번 테이블이 좀 더러운데?")
print(response2)

response3 = validate_alba_task_discriminator("1번 테이블 생일이래")
print(response3)

response4 = validate_alba_task_discriminator("2번 핑키 뭐가 보이는지 설명해")
print(response4)

response5 = validate_alba_task_discriminator("3번 핑키 오늘 주식 현황 알려줘")
print(response5)


greetings
cleaning
birthday
['cleaning', 'serving', 'birthday', 'emergency', 'camera on', 'greetings']에 존재하지 않습니다.
None
['cleaning', 'serving', 'birthday', 'emergency', 'camera on', 'greetings']에 존재하지 않습니다.
None


In [33]:
response6 = validate_alba_task_discriminator("2번 핑키 뭐가 보이는지 설명해")
print(response6)

response7 = validate_alba_task_discriminator("2번 핑키 카메라 켜")
print(response7)

response8 = validate_alba_task_discriminator("3번 핑키 오늘 주식 현황 알려줘")
print(response8)

['cleaning', 'serving', 'birthday', 'emergency', 'camera on', 'greetings']에 존재하지 않습니다.
None
camera on
['cleaning', 'serving', 'birthday', 'emergency', 'camera on', 'greetings']에 존재하지 않습니다.
None


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI

from dotenv import load_dotenv
import time

load_dotenv()

# 메모리 생성 (기록 저장용)
memory = ConversationBufferMemory(memory_key="chat_history", input_key="user_query")

pinky_prompt = generate_pinky_response()
pinky_template = PromptTemplate(template = pinky_prompt, input_variables=["chat_history", "user_query"])

# 객체 생성
llm = ChatOpenAI(
    temperature=0.7,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048,  # 최대 토큰수
    model_name="gpt-4o-mini",  # 모델명
)

chain = LLMChain(
    llm=llm,
    prompt=pinky_template,
    memory=memory,
    verbose=True
)

In [ ]:
# 대화 TCP 전송 테스트

import json
import socket
import logging

tcp_host = "192.168.0.156"
tcp_port = 8001

# get_logger 정의
def get_logger():
    logger = logging.getLogger("TCPClient")
    if not logger.handlers:
        logger.setLevel(logging.INFO)
        handler = logging.StreamHandler()
        formatter = logging.Formatter('[%(asctime)s] %(levelname)s - %(message)s')
        handler.setFormatter(formatter)
        logger.addHandler(handler)
    return logger

def send_tcp_data(data):
    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock: #socket.SOCK_STREAM : TCP 통신
            sock.connect((tcp_host, tcp_port))
            sock.sendall(json.dumps(data).encode('utf-8'))
            response = sock.recv(1024).decode('utf-8')
            get_logger().info(f"서버 응답: {response}")
    except Exception as e:
        get_logger().error(f"TCP 연결 실패: {e}")

conv_1 = {
    "id": 10,
    "type": "CHATBOT",
    "question": "안녕 핑키!",
    "answer": "😊 Nice to meet you! As PINGKI, I'm here to assist the restaurant owner and provide helpful responses. What can I help you with today? 🤔",
}        

send_tcp_data(conv_1)

[2025-04-25 14:43:48,367] INFO - 서버 응답: ERROR
